In [ ]:
!pip install opendatasets --upgrade

In [ ]:
import opendatasets as od
import os
import cv2
import random
import matplotlib.pyplot as plt
import pickle
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img

In [ ]:
od.download('https://www.kaggle.com/competitions/vision-verse/data')

Skipping, found downloaded files in "./vision-verse" (use force=True to force download)


In [ ]:
train_data_dir = '/content/vision-verse/data/train';
Categories = ['crownandrootrot','healthywheat','leafrust','wheatloosesmut']


In [ ]:
# import imageio as iio
# count=0
# for c in Categories:
#   dirr=train_data_dir+"/"+c
#   for l in os.listdir(dirr):
#     imgg=iio.imread(dirr+"/"+l)
#     if l[-1]=='f' and l[-2]=='i' and l[-3]=='g':
#       print(l)


In [ ]:
# import imageio as iio
# count=0

# for c in Categories:
#   dirr=train_data_dir+"/"+c
#   for l in os.listdir(dirr):
#     imgg=iio.imread(dirr+"/"+l)
#     imgg=np.resize(imgg, (32, 32, 3))
#     data.append([imgg,Categories.index(c)])
  


In [ ]:
# data=np.array(data)

In [ ]:
IMG_SIZE = 224
data=[]
i=0
for category in Categories:
  folder = os.path.join(train_data_dir,category)
  label = Categories.index(category)
  for img in os.listdir(folder):
    img_path = os.path.join(folder,img)
    img_arr = cv2.imread(img_path)
   # print(img_arr)
    # try:
    #        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
    #        print(img.shape)
    # except:
    #     print('err')
    # print(category,i)
    try:
        # image = cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH), interpolation=cv2.INTER_AREA)
        img_arr = cv2.resize(img_arr,(IMG_SIZE,IMG_SIZE))
        # print(img_arr.shape)
    except:
        print(img_arr,2)
        continue
   # print(img_path)
    img_arr = cv2.resize(img_arr,(IMG_SIZE,IMG_SIZE))
    data.append([img_arr,label])
    i=i+1

None 2
None 2
None 2
None 2
None 2
None 2
None 2
None 2
None 2
None 2
None 2
None 2
None 2
None 2
None 2
None 2
None 2
None 2
None 2
None 2


In [ ]:
random.shuffle(data)
batch_size = 32

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary',
    shuffle = True,
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary',
    shuffle = True,
    subset='validation') # set as validation data

Found 1813 images belonging to 4 classes.
Found 451 images belonging to 4 classes.


In [ ]:
train_generator


In [ ]:
feature = []
label = []

for features,labels in data:
  feature.append(features)
  label.append(labels)
feature = np.array(feature)
label = np.array(label)
feature = feature.astype('float')
feature=feature/255

feature.shape

(2543, 224, 224, 3)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [ ]:

import tensorflow as tf


model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=16, kernel_size=(3,3), activation='relu',input_shape=(224, 224, 3)),
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.BatchNormalization(axis=-1),

    tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.BatchNormalization(axis=-1),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation='relu'),
    tf.keras.layers.BatchNormalization() ,
    tf.keras.layers.Dropout(rate=0.5),

    tf.keras.layers.Dense(4,activation='softmax')

])
learning_rate = 0.001
epochs=2
opt= tf.keras.optimizers.Adam(learning_rate=learning_rate , decay=learning_rate/(epochs*0.5))
model.compile(optimizer='adam',loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [ ]:
model.fit(feature, label, epochs=10, validation_split=0.1)

Epoch 1/10
72/72 [==============================] - 23s 321ms/step - loss: 0.0448 - accuracy: 0.9895 - val_loss: 0.2583 - val_accuracy: 0.9451
Epoch 2/10
72/72 [==============================] - 23s 318ms/step - loss: 0.0434 - accuracy: 0.9908 - val_loss: 0.4298 - val_accuracy: 0.8980
Epoch 3/10
72/72 [==============================] - 23s 317ms/step - loss: 0.0398 - accuracy: 0.9917 - val_loss: 0.4395 - val_accuracy: 0.9020
Epoch 4/10
72/72 [==============================] - 23s 317ms/step - loss: 0.0298 - accuracy: 0.9926 - val_loss: 0.3156 - val_accuracy: 0.9373
Epoch 5/10
72/72 [==============================] - 23s 317ms/step - loss: 0.0322 - accuracy: 0.9917 - val_loss: 0.3391 - val_accuracy: 0.9333
Epoch 6/10
72/72 [==============================] - 23s 318ms/step - loss: 0.0278 - accuracy: 0.9917 - val_loss: 0.2337 - val_accuracy: 0.9333
Epoch 7/10
72/72 [==============================] - 23s 317ms/step - loss: 0.0419 - accuracy: 0.9926 - val_loss: 0.4499 - val_accuracy: 0.9059

In [ ]:
dir = '/content/vision-verse/data/test'
df = pd.read_csv('/content/vision-verse/data/sample_submission.csv')
print(df)

                       path            label
0      data/test/000146.jpg  crownandrootrot
1      data/test/000191.jpg  crownandrootrot
2      data/test/000185.jpg  crownandrootrot
3      data/test/000218.jpg  crownandrootrot
4      data/test/000224.jpg  crownandrootrot
...                     ...              ...
1096   data/test/000995.jpg  crownandrootrot
1097  data/test/0001058.jpg  crownandrootrot
1098   data/test/000981.jpg  crownandrootrot
1099  data/test/0001064.jpg  crownandrootrot
1100  data/test/0001070.jpg  crownandrootrot

[1101 rows x 2 columns]


In [ ]:
testing = []
c=0
count =0
for img in os.listdir(dir):
    img_path = os.path.join(dir,img)
    img_arr = cv2.imread(img_path)
    try:
        # image = cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH), interpolation=cv2.INTER_AREA)
        img_arr = cv2.resize(img_arr,(IMG_SIZE,IMG_SIZE))
        # img_arr = cv2.resize(img_arr,(IMG_SIZE,IMG_SIZE))
        # 'crownandrootrot','healthywheat','leafrust','wheatloosesmut']
        img_arr=img_arr/255
        img_arr = np.expand_dims(img_arr,axis=0)
        val = model.predict(img_arr)
        print(val)
        if(val[0][0]>val[0][1] and val[0][0]>val[0][2] and val[0][0]>val[0][3]):
          df.loc[df['path']=='data/test/'+str(img),label]='crownandrootrot'
        elif(val[0][1]>val[0][0] and val[0][1]>val[0][2] and val[0][1]>val[0][3]):
          df.loc[df['path']=='data/test/'+str(img),label]='healthywheat'
        elif(val[0][2]>val[0][0] and val[0][2]>val[0][1] and val[0][2]>val[0][3]):
          df.loc[df['path']=='data/test/'+str(img),label]='leafrust'
        elif(val[0][3]>val[0][0] and val[0][3]>val[0][2] and val[0][3]>val[0][1]):
          df.loc[df['path']=='data/test/'+str(img),label]='wheatloosesmut'
        count=count+1
        print(count)
        print(img_arr.shape)
    except:
        df.loc[df['path']=='data/test/'+str(img),label]='crownandrootrot'
        c=c+1
        print(c)
        continue
    # img_arr = cv2.resize(img_arr,(IMG_SIZE,IMG_SIZE))
    # img_arr=img_arr/255
    # img_arr = np.expand_dims(img_arr,axis=0)
    # val = model.predict(img_arr)
    # c=c+1
    # print(c)
    # print(val)
    # if(val[0][0]>val[0][1]):
    #   df.loc[df['path']=='data/test/' + str(img),label]='fake'
    # else:
    #   df.loc[df['path']=='data/test/' + str(img),label]='real'

[[2.4177729e-05 9.9939430e-01 5.6163815e-04 1.9905345e-05]]
1
(1, 224, 224, 3)
[[9.9983335e-01 1.3391891e-05 9.6386953e-05 5.6915673e-05]]
2
(1, 224, 224, 3)
[[9.9987459e-01 8.0482278e-05 4.4816286e-05 6.5605548e-08]]
3
(1, 224, 224, 3)
[[1.7779639e-05 5.5041652e-05 2.2909365e-04 9.9969816e-01]]
4
(1, 224, 224, 3)
[[1.5325080e-05 2.7865669e-04 9.9964213e-01 6.3823092e-05]]
5
(1, 224, 224, 3)
[[4.1225110e-04 6.3509542e-05 9.8722827e-01 1.2295936e-02]]
6
(1, 224, 224, 3)
[[2.5554935e-05 9.9995911e-01 1.4456862e-05 8.7127103e-07]]
7
(1, 224, 224, 3)
[[3.0056329e-04 1.4864708e-03 2.7307885e-04 9.9793988e-01]]
8
(1, 224, 224, 3)
[[1.2808491e-05 9.9995196e-01 2.9542487e-05 5.7967809e-06]]
9
(1, 224, 224, 3)
[[9.9998009e-01 8.3186551e-06 1.8334531e-06 9.7293532e-06]]
10
(1, 224, 224, 3)
[[1.8233975e-05 9.9658585e-01 3.3952037e-03 7.2252590e-07]]
11
(1, 224, 224, 3)
[[5.6151235e-05 9.9991548e-01 2.3469367e-05 4.9328078e-06]]
12
(1, 224, 224, 3)
[[9.9383318e-01 3.0543084e-05 5.6674825e-03 4.687

In [ ]:
df.to_csv('result2.csv', index=False)